In [ ]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt

from src.utils import print_yaml, bootstrap_sampling, interpolate_unique
from src.save_load import loadAudit, savePlot
from LeakPro.leakpro.attacks.mia_attacks.lira import lira_vectorized

In [ ]:
# -------------------------- #
#   Load the audit signals   #
# -------------------------- #
# Enter the name of the sub directory within the audit_signals directory that is to be analyzed
audit_dir = "20251117_b2919d4746"
metadata, rescaled_target_logits, rescaled_sm_logits, sm_in_mask, audit_data_indices = loadAudit(audit_dir)
    
audit_cfg = metadata["auditCfg"]

In [ ]:
# --------------- #
#   Verify data   #
# --------------- #
print_yaml(metadata)
print(f"\n Target logits shape: {rescaled_target_logits.shape}")
print(f"\n SM logits shape: {rescaled_target_logits.shape} SM in mask shape: {sm_in_mask.shape}")

In [ ]:
# --------------------------- #
#   Setup for bootstrapping   #
# --------------------------- #
# Shadow models used
num_sm = audit_cfg["audit"]["attacklist"]["0"]["num_shadow_models"]
# Samples for bootstrapping
no_samples = 100
random_target = True

In [ ]:
# ---------------------- #
#   Bootstrap sampling   #
# ---------------------- #

samples = bootstrap_sampling(num_sm, no_samples, rescaled_sm_logits, sm_in_mask, rescaled_target_logits, target_inmask, vec_mia_fun=lira_vectorized)

unique_fpr = sorted(np.unique(np.concat([fpr for (fpr, tpr) in samples])))

ig, axs = plt.subplots(1,2,figsize=(12,4))
axs[0].plot(unique_fpr, unique_fpr, 'r--', label="random")
axs[1].loglog(unique_fpr, unique_fpr, 'r--', label="random")
for i in [0,1]:
    for (fpr, tpr) in samples:
        axs[i].plot(fpr, tpr)
    axs[i].set_xlabel("FPR")
    axs[i].set_ylabel("TPR")
    axs[i].legend()
plt.tight_layout()
plt.show()

plot_title = "cifar10_resnet18_baseline_tpr-fpr"

In [ ]:
# ---------------------- #
#   Bootstrap sampling   #
# ---------------------- #
interp_tpr = np.array([interpolate_unique(unique_fpr, fpr, tpr) for (fpr, tpr) in samples])

fig, axs = plt.subplots(1,2,figsize=(12,4))
axs[0].plot(fpr, fpr, 'r--', label="random")
axs[1].loglog(fpr, fpr, 'r--', label="random")
for i in [0,1]:
    axs[i].plot(unique_fpr,np.mean(interp_tpr,axis=0))
    axs[i].fill_between(unique_fpr,np.percentile(interp_tpr,2.5,axis=0),np.percentile(interp_tpr,97.5,axis=0), alpha=0.5)
    axs[i].set_xlabel("FPR")
    axs[i].set_ylabel("TPR")
    axs[i].legend()
plt.tight_layout()
plt.show()

plot_title = "cifar10_resnet18_baseline_tpr-fpr_confidence"
savePlot(fig, plot_title, audit_dir)